# Interacting with GPIO from MicroBlaze

In [2]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")

In [11]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

In [13]:
write_gpio(0, 2)
read_gpio(1)

pin value must be 0 or 1

1

# Multi-tasking with MicroBlaze 

In [5]:
base = BaseOverlay("base.bit")

In [6]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODA
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Multitasking the microblaze for a simple function
int add(int a, int b){
    return a + b;
}

In [7]:
val = 1
write_gpio(0, val)
read_gpio(1)

1

In [8]:
add(2, 30)

32

# Lab work
Use the code from the second cell as a template and write a code to use two pins (0 and 1) for send and two pins (2 and 3) for receive. You should be able to send 2bits (0~3) over GPIO. You'll need to hardwire from the send pins to the receive pins.

In [3]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [4]:
%%microblaze base.PMODB
#include "gpio.h"
#include "pyprintf.h"

/**
 * Writes a 2-bit value to GPIO pins 0 and 1
 * Pin configuration:
 * - Pin 0: Least Significant Bit (LSB)
 * - Pin 1: Most Significant Bit (MSB)
 * 
 * @param value: Integer between 0-3 representing the 2-bit value to write
 * @return: Returns the written value, or 4 if input was invalid
 */
unsigned int write_2bits(unsigned int value) {
    // Input validation - ensure value is 2 bits (0-3)
    if (value > 3) {
        pyprintf("Value must be between 0 and 3\n");
        return 4;  // Return error code
    }
    
    // Extract individual bits from the input value
    unsigned int bit0 = value & 0x1;        // Extract LSB using bitwise AND with 1
    unsigned int bit1 = (value >> 1) & 0x1; // Shift right by 1 and extract MSB
    
    // Initialize GPIO pins for output
    gpio pin0_out = gpio_open(0);  // LSB pin
    gpio pin1_out = gpio_open(1);  // MSB pin
    
    // Configure pins as outputs
    gpio_set_direction(pin0_out, GPIO_OUT);
    gpio_set_direction(pin1_out, GPIO_OUT);
    
    // Write values to pins
    gpio_write(pin0_out, bit0);  // Write LSB to pin 0
    gpio_write(pin1_out, bit1);  // Write MSB to pin 1
    
    return value;  // Return the successfully written value
}

/**
 * Reads a 2-bit value from GPIO pins 2 and 3
 * Pin configuration:
 * - Pin 2: Least Significant Bit (LSB)
 * - Pin 3: Most Significant Bit (MSB)
 * 
 * @return: Integer between 0-3 representing the read 2-bit value
 */
unsigned int read_2bits() {
    // Initialize GPIO pins for input
    gpio pin2_in = gpio_open(2);  // LSB pin
    gpio pin3_in = gpio_open(3);  // MSB pin
    
    // Configure pins as inputs
    gpio_set_direction(pin2_in, GPIO_IN);
    gpio_set_direction(pin3_in, GPIO_IN);
    
    // Read values from pins
    unsigned int bit0 = gpio_read(pin2_in);  // Read LSB from pin 2
    unsigned int bit1 = gpio_read(pin3_in);  // Read MSB from pin 3
    
    // Combine bits into final value:
    // Shift MSB left by 1 position and combine with LSB using bitwise OR
    return (bit1 << 1) | bit0;
}

In [5]:
# Test writing different values:
write_2bits(0)   # Write 00 (binary) = 0 (decimal)
write_2bits(1)   # Write 01 (binary) = 1 (decimal)
write_2bits(2)   # Write 10 (binary) = 2 (decimal)
write_2bits(3)   # Write 11 (binary) = 3 (decimal)

# Read the current value
read_2bits()     # Read back the value from pins 2 and 3

3